In [303]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit
import math

# Some helper functions:
def sigmoid(z):
    #using the numerically stable expit function from scipy to avoid overflow errors
    return expit(z)

def classifyLogistic(v):
    """ 
    v is a vector (numpy ndarray) of probabilities
    classify each example (row) 1 if >=0.5 and 0 otherwise
    """
    v[v>=0.5] = 1
    v[v<0.5] = 0
    return v

def classifyLDA(v):
    """ 
    v is a vector (numpy ndarray) of log-odds ratios
    classify each example (row) 1 if >0 and 0 otherwise
    """
    v[v>0] = 1
    v[v<=0] = 0
    return v

#Data Cleaning
winedf = pd.read_csv("winequality-red.csv", sep=';')
wineData = winedf.to_numpy()
wineData[:,-1] = (wineData[:,-1]>=6).astype(int) #convert 6+ to 1 and <5 to 0
wineData = wineData/wineData.max(axis=0) #normalize data
bcdf = pd.read_csv("breast-cancer-wisconsin.data", sep=',', header=None)
bcdf.columns = ['Sample Code Number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
                'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
                'Normal Nucleoli', 'Mitoses', 'Class']
bcdf.replace('?', np.NaN, inplace=True)
bcdf.dropna(inplace=True)
bcdf.drop(['Sample Code Number'], axis=1, inplace=True)
bcData = bcdf.to_numpy().astype(float)
bcData[:,-1] = (bcData[:,-1]>3).astype(int) #change 2/4 last column to 0/1 labels
bcData = bcData/bcData.max(axis=0) #normalize data


def train_test_split(dataset, ratio=0.2):
    """
    split dataset into training and test subsets
    test set size will be ratio * dataset size (default = 20% of total size)
    returns data_train, data_test
    """
    dataset_size = dataset.shape[0]
    test_set_size = math.floor(ratio * dataset_size)
    training_set_size = dataset_size - test_set_size
    data_train = dataset[0:training_set_size, :]
    data_test = dataset[test_set_size:, :]
    return data_train, data_test

class LDA:
    """Implementing an LDA (Linear Discriminant Analysis) model from scratch"""

    def __init__(self, data):
        """
        Constructor to create a new LDA instance

        data        = matrix (numpy ndarray) of dataset with labels as the last column
        X           = matrix (numpy ndarray) of dataset with labels (last column) removed
        y           = vector (numpy ndarray) of labels (last column of data)
        m           = total number of training examples (= number of rows of X or y)
        n           = number of features = num of cols - 1 (last column is labels)
        X1          = (numpy ndarray) rows of data that belong to class 1
        X0          = (numpy ndarray) rows of data that belong to class 0

        """
        self.data = data
        self.X = data[:,:-1]
        self.y = data[:,-1][:, np.newaxis]
        self.m = data.shape[0]
        self.n = data.shape[1] - 1
        self.X1 = self.X[data[:,-1]==1]
        self.X0 = self.X[data[:,-1]==0]

    # Model implementation begins below:     
    def fit(self):
        """
        Fit training data
        Calculates log odds ratio (by calculating mean and covariance) using training data.

        num0        = number of training examples in class 0 (number of rows of X0)
        num1        = number of training examples in class 1 (number of rows of X1)
        py0         = P(y=0) calculated by num0 / (num0 + num1)
        py1         = P(y=1) calculated by num1 / (num0 + num1)
        mean1       = array of mean of columns in X1
        mean0       = array of mean of columns in X0
        sigma       = shared covariance matrix for the dataset

        """

        self.num1 = self.X1.shape[0] 
        self.num0 = self.X0.shape[0]
        self.py0 = self.num0/self.m
        self.py1 = self.num1/self.m
        self.mean1 = np.mean(self.X1, axis=0)[:,np.newaxis]
        self.mean0 = np.mean(self.X0, axis=0)[:,np.newaxis]
        self.sigma = (np.cov(self.X1.T) + np.cov(self.X0.T)) / (self.m - 2)


    def predict(self, X_test):
        """
        Given a trained model, predict labels for new data X_test (which is a mxn matrix),
        predict returns a m-dimensional vector of 0/1 predicted labels

        logodds_vector = vector of predicted log odds for every row in X_test
        """
        sigma_inv = np.linalg.inv(self.sigma)
        term1 = np.log(self.num1/self.num0) #no need to include the denominator as it cancels out
        term2 = 0.5*(self.mean1.T @ sigma_inv @ self.mean1)
        term3 = 0.5*(self.mean0.T @ sigma_inv @ self.mean0)
        term4 = X_test @ sigma_inv @ (self.mean1-self.mean0)

        logodds_vector = term1 - term2 + term3 + term4

        predicted_labels = classifyLDA(logodds_vector)
        return predicted_labels

    def evaluate_acc(self, y_predict, y_test):
        """
        Check the accuracy of the predictions calculated by the predict method of the model
        Returns a percentage accuracy (float)
        """

        test_set_size = y_test.shape[0]
        numErrors = np.sum( np.abs(y_predict - y_test) )
        errorPercentage = (numErrors/test_set_size) * 100
        accuracy = 100 - errorPercentage
        return accuracy

In [304]:
model = LDA(bcData)

In [305]:
model.fit()

In [306]:
X_test = bcData[:,:-1]

In [307]:
y_predict = model.predict(X_test)

In [308]:
model.evaluate_acc(y_predict, bcData[:,-1][:,np.newaxis])

96.63250366032212